In [1]:
import torch
from torchvision import transforms
import torch.nn as nn
import timm

c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.41427545, 0.40396059, 0.39235218], 
                         std=[0.17746354, 0.17282704, 0.16986775])
])

In [4]:
class custom_model(nn.Module):
    def __init__(self, backbone, num_classes):
        super().__init__()

        if isinstance(backbone, str):
            self.backbone = timm.create_model(backbone, pretrained=True, num_classes=0)
        elif isinstance(backbone, nn.Module):
            self.backbone = backbone
        else:
            raise

        with torch.no_grad():
            dummy = torch.randn(1, 3, 224, 224)
            num_features = self.backbone(dummy).shape[1]

        self.head = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.GELU(),
            nn.Linear(512, 128),
            nn.GELU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x, return_features=False):
        features = self.backbone(x)

        if return_features:
            return self.head(features), features
        
        return self.head(features)
    
    def forward_features(self, x):
        return self.backbone(x)

In [6]:
class custom_model(nn.Module):
    def __init__(self, backbone, num_classes):
        super().__init__()

        if isinstance(backbone, str):
            self.backbone = timm.create_model(backbone, pretrained=True, num_classes=0)
        elif isinstance(backbone, nn.Module):
            self.backbone = backbone
        else:
            raise

        with torch.no_grad():
            dummy = torch.randn(1, 3, 224, 224)
            num_features = self.backbone(dummy).shape[1]

        self.head = nn.Sequential(
            nn.Linear(num_features, 2048),
            nn.GELU(),
            nn.Linear(2048, 2048),
            nn.Linear(2048, 1024),
            nn.Linear(1024, 256),
            nn.GELU(),
            nn.Linear(256, num_classes)
        )

    def forward(self, x, return_features=False):
        features = self.backbone(x)

        if return_features:
            return self.head(features), features
        
        return self.head(features)
    
    def forward_features(self, x):
        return self.backbone(x)

In [ ]:
# _backbone = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=0)

# model = custom_model(_backbone, 7).to('cuda')

model = torch.load('model_4.pt', weights_only=False)
model.eval()

custom_model(
  (backbone): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (patch_drop): Identity()
    (norm_pre): Identity()
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (q_norm): Identity()
          (k_norm): Identity()
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): Identity()
        (drop_path1): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate='non

In [29]:
import pandas as pd

In [31]:
test_csv = pd.read_csv(r'C:\Users\user\Downloads\open\open\test.csv')
test_csv

,ID,img_path
0,TEST_00000,./test/TEST_00000.jpg
1,TEST_00001,./test/TEST_00001.jpg
2,TEST_00002,./test/TEST_00002.jpg
3,TEST_00003,./test/TEST_00003.jpg
4,TEST_00004,./test/TEST_00004.jpg
...,...,...
95001,TEST_95001,./test/TEST_95001.jpg
95002,TEST_95002,./test/TEST_95002.jpg
95003,TEST_95003,./test/TEST_95003.jpg
95004,TEST_95004,./test/TEST_95004.jpg


In [32]:
_label_list = ['Andesite','Basalt','Etc','Gneiss','Granite','Mud_Sandstone','Weathered_Rock']

In [ ]:
import os
from PIL import Image

results = []

for row in test_csv.itertuples():
    true_path = os.path.join(rf'C:\Users\user\Downloads\open\open\test\{row.ID}.jpg')

    _image = Image.open(true_path)
    _tensor_image = transform(_image)

    _result = model(_tensor_image.to('cuda').unsqueeze(0))
    _result = torch.argmax(_result, dim=1)
    results.append(_label_list[_result[0]])

In [ ]:
test_csv['rock_type'] = results

In [ ]:
test_csv = test_csv[['ID', 'rock_type']]

In [ ]:
pd.DataFrame.to_csv(test_csv, 'result_7.csv', index=False)

In [65]:
result_dict = {}

for item in results:
    if result_dict.get(item) == None:
        result_dict[item] = 0

    result_dict[item] += 1

result_dict

{'Mud_Sandstone': 16596,
 'Etc': 5937,
 'Granite': 25370,
 'Gneiss': 20490,
 'Andesite': 7284,
 'Weathered_Rock': 8177,
 'Basalt': 11152}